In [1]:
import sys
import os

# Add the root directory to the Python environment
root_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_dir)

In [2]:
import pickle
import networkx as nx
import gzip
import matplotlib.pyplot as plt
import pandas as pd
from src import uniswap, uniprice_2
from src.graph import create_graph
from src.algo_per_coin import safe_literal_eval

In [3]:
top_coins_count_list = [100, 500, 1000, 5000, 10000, 50000, 100000]

In [4]:
main_snapshot_block_number = 22816000

In [5]:
df = pd.read_csv(f"../data/snapshots/snapshot_{main_snapshot_block_number}.csv.gz", compression="gzip")

df['other'] = df['other'].apply(safe_literal_eval)
df['token0Price'] = pd.to_numeric(df['token0Price'], errors='coerce')
df['token1Price'] = pd.to_numeric(df['token1Price'], errors='coerce')
df['totalValueLockedUSD'] = pd.to_numeric(df['totalValueLockedUSD'], errors='coerce')

df = df.dropna(subset=['other', 'token0Price', 'token1Price'])


# Check if 'feeTier' column exists, if not, create it
if 'feeTier' not in df.columns:
    df['feeTier'] = df.apply(
        lambda row: int(row['other'].get('feeTier', 0)) if row['version'] in ['v3', 'v4'] 
        else 3000 if row['version'] == 'v2' 
        else None, 
        axis=1
    )
    df['feeTier'] = df['feeTier'].apply(float)

df.head()

/tmp/ipykernel_2477/818620695.py:1: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"../data/snapshots/snapshot_{main_snapshot_block_number}.csv.gz", compression="gzip")


,createdAtBlockNumber,id,token0Price,token1Price,liquidity,totalValueLockedUSD,other,token0_decimals,token0_id,token0_name,token0_symbol,token1_decimals,token1_id,token1_name,token1_symbol,version,block_number,feeTier
0,13844117.0,0x76b36afc7f1ef4bbf3301b602c9f6ca2ff6c3bd8,1.660335e+17,0.000000e+00,934796603.601451,8269.914769,"{'totalSupply': '26538.175287407292876187', 'r...",9.0,0x5d8038644608d1f849ed2c6863a2ea667e53371a,INUGAMI,INUGAMI,18.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,v2,22815924.0,3000.0
1,17787279.0,0x9fe0f9ef9c17a1afa2c29078ec3323d4cf7a9ddc,2.100182e-09,4.761492e+08,38571.818436,8269.085571,"{'totalSupply': '37947.331922020551982986', 'r...",18.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18.0,0xcd3b1f5b3bad4ef75f2bc840b005b55f7afbe4aa,X Æ A-12 Musk,LilX,v2,22815924.0,3000.0
2,22460490.0,0x6b8faec9d9e4c5d1b3f74b7640840544d378f6a4,3.441434e+07,2.905765e-08,9317.689528,8268.901696,"{'totalSupply': '0.282842712474618009', 'reser...",9.0,0xbbb4099798b46a82283ef3b1aa3007cf4c17870f,Tomato Frog,TFROG,18.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,v2,22815924.0,3000.0
3,20428614.0,0x537bb3598a20937c4bfdb413c6d46c13cd0a5dd6,1.838279e+11,5.439800e-12,639462.487731,8268.738322,"{'totalSupply': '18.345353635185121808', 'rese...",9.0,0x3be7bbe6d61eda1524a904e5dde46a22afeb35e8,COCONUT,COCONUT,18.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,v2,22815924.0,3000.0
4,17137011.0,0xb5b8aee2af5b8d485bac4ed80c71dd8594096dfa,1.430884e+14,6.900000e-15,19508999.450781,8267.930282,"{'totalSupply': '577.061521850140341555', 'res...",9.0,0x80b7a77d280a4ef3365cd34316231b7ee6e8b0c5,Pepe Tate,PATE,18.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,v2,22815924.0,3000.0


In [6]:
len(set(df.token0_id.unique()).union(set(df.token1_id.unique())))

423870

In [7]:
df_top_tokens = uniswap.get_top_tokens(df, max(top_coins_count_list))
df_top_tokens.head(20)

,token_id,token_symbol
349195,0xd286511359a0b7f3d77d96a4fce06a9226a24353,unknown
261273,0x9cc32f63850a3d9a64d613a4fa7b1c7d8993c054,CRYPT
251083,0x96884fcaac082db4b32601ada5b177fd6cbffa88,ZKLK
100230,0x3cb48aeb3d1abadc23d2d8a6894b3a68338381c2,PALAI
139735,0x5483dc6abda5f094865120b2d251b5744fc2ecb5,TPAD
409433,0xf7498c98789957f4ee53b3e37ff5b7ef8a6cfc7b,0xDEV
36806,0x160de4468586b6b2f8a92feb0c260fc6cfc743b1,ease.org
388241,0xea5edef1c6ed1be1bcba4617a1c5a994e9018a43,ez-cvxsteCRV
319217,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH
94710,0x395c8db957d743a62ac3aaaa4574553bcf2380b3,UETH


In [8]:
G = create_graph(df, apply_fee=False, remove_low_degree_nodes=False, remove_multi_edges=True, fake_fee=0)
print(len(G.nodes), len(G.edges))

423870 868738


In [9]:
# save graph object to file
graph_path = os.path.join("/workspaces/uniswap-v3-crawler/data/graphs", f"graph_{main_snapshot_block_number}.gpickle")

In [10]:
with gzip.open(graph_path + '.gz', 'wb') as f:
    pickle.dump(G, f)

In [11]:
with gzip.open(graph_path + '.gz', 'rb') as f:
    G = pickle.load(f)

In [12]:
# G_sub = G.subgraph(df_top_tokens.token_id.head(100000))
# nx.approximation.treewidth_min_degree(nx.Graph(G_sub.to_undirected()))[0] 

In [13]:
# up = uniprice_2.UniPrice()
# up.accept_graph(G_sub)